In [1]:
import os
from pathlib import Path
import sys

# !IMPORTANT!, else Hydra isnt working !
current_path = os.getcwd()
sys.path.insert(0, Path(current_path).absolute().parents[0].as_posix())

import cv2
import tensorflow_datasets as tfds
import tensorflow as tf
import tqdm
import mediapy
import numpy as np
import torch
from uha import make_pytorch_oxe_iterable_dataset, get_octo_dataset_tensorflow, get_single_dataset_tensorflow
from torchvision.transforms.functional import convert_image_dtype
import omegaconf
from omegaconf import OmegaConf

/home/marcelr/miniforge3/envs/uha_test_policy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-18 10:20:01.584256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 10:20:01.584314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 10:20:01.586433: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-18 10:20:01.595717: I tensorflow/core/platform/cpu_feature_guard.cc:182] This

## Load Data
Next, we will load a trajectory from the Bridge dataset. We will use the one defined in uha/data/oxe/oxe_dataset_configs.py.

In [2]:
cfg = omegaconf.OmegaConf.load("../uha/data/conf/uha_default_load_config.yaml")
del cfg.interleaved_dataset_cfg.frame_transform_kwargs.image_augment_kwargs # comment out to get augmented data
cfg_transforms = omegaconf.OmegaConf.load("../uha/data/conf/transforms/oxe_no_remapping.yaml")

In [4]:
# dataset = get_octo_dataset_tensorflow(cfg, train=True)
dataset = get_single_dataset_tensorflow(cfg, train=True) # .repeat().unbatch()
is_single_dataset = True
batch_size = 128
# create Pytorch Train Dataset
dataloader = make_pytorch_oxe_iterable_dataset(dataset, train=True, batch_size=batch_size, transform_dict=cfg_transforms, num_workers=0, pin_memory=True, is_single_dataset=is_single_dataset, main_process=False)
# print("len in colab", len(dataloader.dataset))
it = iter(dataloader)
batch = next(it)
batch = next(it)
batch = next(it)
images, image_secondary, image_wrist = [], [], []
for i in range(batch_size):
  images.append(batch["observation"]["image_primary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  image_secondary.append(batch["observation"]["image_secondary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  image_wrist.append(batch["observation"]["image_wrist"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  print("action", batch["action"][i, :, 0, :])

mediapy.show_video(images, fps=10)
mediapy.show_video(image_secondary, fps=10)
mediapy.show_video(image_wrist, fps=10)

########################################
constructing single val dataset: toto
########################################


2024-07-18 10:25:05.426534: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
1003it [08:27,  1.98it/s]
2024-07-18 10:33:39.320505: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-18 10:35:06.437874: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 1322651484160 bytes after encountering the first element of size 322912960 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


action tensor([[ 0.2076,  0.5271,  0.0850,  0.4184, -0.0905, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2084,  0.5271,  0.0850,  0.4186, -0.0888, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2094,  0.5271,  0.0850,  0.4187, -0.0869, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2105,  0.5271,  0.0850,  0.4187, -0.0852, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2119,  0.5272,  0.0851,  0.4187, -0.0837, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2129,  0.5271,  0.0852,  0.4187, -0.0828, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2139,  0.5271,  0.0854,  0.4187, -0.0813, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2148,  0.5271,  0.0857,  0.4186, -0.0790, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2161,  0.5271,  0.0859,  0.4185, -0.0760, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2168,  0.5271,  0.0860,  0.4184, -0.0748, -0.5001,  0.9229,  0.0000]])
action tensor([[ 0.2177,  0.5270,  0.0862,  0.4184, -0.0746, -0.5001,  0.9228,  0.0000]])
action ten

This browser does not support the video tag.
